<a href="https://colab.research.google.com/github/ahelmasri87/aaa/blob/main/Bert_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#BERT embeddings
sbs_df.columns = sbs_df.columns.str.strip().str.lower()
cpt_df.columns = cpt_df.columns.str.strip().str.lower()
achi_df.columns = achi_df.columns.str.strip().str.lower()
sbs_df["service description"] = sbs_df["service description"].str.lower()
cpt_df["service description"] = cpt_df["service description"].str.lower()
achi_df["service description"] = achi_df["service description"].str.lower()
sbs_df = pd.read_excel("/content/sbs_codes.xlsx", header=0)
cpt_df = pd.read_excel("/content/cpt_codes.xlsx", header=0)
achi_df = pd.read_excel("/content/achi_codes.xlsx", header=0)
from sentence_transformers import SentenceTransformer


import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util

# Load BERT model for embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")

# Standardizing column names
sbs_df.columns = sbs_df.columns.str.strip().str.lower()
cpt_df.columns = cpt_df.columns.str.strip().str.lower()
achi_df.columns = achi_df.columns.str.strip().str.lower()

# Convert descriptions to lowercase for uniformity
sbs_df["service description"] = sbs_df["service description"].str.lower()
cpt_df["service description"] = cpt_df["service description"].str.lower()
achi_df["service description"] = achi_df["service description"].str.lower()

# Compute embeddings for each dataset
sbs_df["embedding"] = sbs_df["service description"].apply(lambda x: model.encode(x, convert_to_tensor=True))
cpt_df["embedding"] = cpt_df["service description"].apply(lambda x: model.encode(x, convert_to_tensor=True))
achi_df["embedding"] = achi_df["service description"].apply(lambda x: model.encode(x, convert_to_tensor=True))



In [ ]:
def get_code_from_description():

    # Step 1: Ask for user input
    user_input = input("Enter the medical service description: ").strip().lower()

    # Step 2: Ask the user to choose the code type
    code_type = input("Which code do you need? (CPT, SBS, ACHI): ").strip().upper()

    # Select the correct dataset
    if code_type == "SBS":
        df = sbs_df
    elif code_type == "CPT":
        df = cpt_df
    elif code_type == "ACHI":
        df = achi_df
    else:
        print("Invalid selection! Please choose CPT, SBS, or ACHI.")
        return

    # Compute user input embedding
    user_embedding = model.encode(user_input, convert_to_tensor=True)

    # Compute similarity scores
    similarity_scores = [util.pytorch_cos_sim(user_embedding, emb)[0].item() for emb in df["embedding"]]

    # Find the best match
    best_match_idx = similarity_scores.index(max(similarity_scores))
    best_match_score = max(similarity_scores)

    # Convert similarity score to percentage
    similarity_percentage = round(best_match_score * 100, 2)

    # Retrieve the best match details
    best_match_description = df.iloc[best_match_idx]["service description"]
    best_match_code = df.iloc[best_match_idx]["code"]

    # Print the outcome
    print("\nBest Match Found:")
    print(f"Code: {best_match_code}")
    print(f"Description: {best_match_description.capitalize()}")
    print(f"Similarity: {similarity_percentage}%")

# Run the function
get_code_from_description()
